# T05 - Motor Trend Car Road Tests

Sara Hernandez Ochoa 


## Cargar datos y librerias 

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
df = pd.read_excel("Motor Trend Car Road Tests.xlsx")
df.head()


,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


## Preparar datos

In [2]:
data = df.drop(columns=['model']).dropna()

for c in data.columns:
    data[c] = pd.to_numeric(data[c], errors='coerce')


## Ejercicio 1.1 (mpg como salida, todo numérico)

In [3]:
X = data.drop(columns=['mpg'])
y = data['mpg']

modelo = LinearRegression().fit(X, y)
print("R² completo:", round(r2_score(y, modelo.predict(X)), 4))
print("Intercepto:", modelo.intercept_)

betas = pd.Series(modelo.coef_, index=X.columns)
betas


R² completo: 0.869
Intercepto: 12.303374155996273


cyl    -0.111440
disp    0.013335
hp     -0.021482
drat    0.787111
wt     -3.715304
qsec    0.821041
vs      0.317763
am      2.520227
gear    0.655413
carb   -0.199419
dtype: float64

El modelo explica 86.9% de la variación en mpg. Factores como mayor peso y caballos reducen rendimiento, mientras que drat, gear y transmisión manual lo aumentan.

## Split 40% train / 60% test + regularización Ridge

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.4, random_state=0)

lin = LinearRegression().fit(X_train, y_train)
print("R² train:", round(r2_score(y_train, lin.predict(X_train)),4))
print("R² test :", round(r2_score(y_test, lin.predict(X_test)),4))

# Ridge con varios alphas
for a in [0.01, 0.1, 1, 10, 100]:
    ridge = Ridge(alpha=a).fit(X_train, y_train)
    print(f"alpha={a:>5}: R² train={r2_score(y_train, ridge.predict(X_train)):.3f},",
          f"R² test={r2_score(y_test, ridge.predict(X_test)):.3f}")


R² train: 0.9992
R² test : 0.2639
alpha= 0.01: R² train=0.999, R² test=0.277
alpha=  0.1: R² train=0.997, R² test=0.289
alpha=    1: R² train=0.987, R² test=0.368
alpha=   10: R² train=0.930, R² test=0.497
alpha=  100: R² train=0.838, R² test=0.489


El modelo sin regularización sobreajusta (train muy alto, test muy bajo). Con mayor alpha, el ajuste en train baja pero la prueba mejora, mostrando que la regularización L2 reduce el sobreajuste y generaliza mejor.

## Ejercicio 1.2 (qsec como salida, todo numérico)

In [5]:
X2 = data.drop(columns=['qsec'])
y2 = data['qsec']

modelo2 = LinearRegression().fit(X2, y2)
print("R² completo:", round(r2_score(y2, modelo2.predict(X2)), 4))

betas2 = pd.Series(modelo2.coef_, index=X2.columns)
betas2


R² completo: 0.8747


mpg     0.069048
cyl    -0.362678
disp   -0.007501
hp     -0.001563
drat   -0.131064
wt      1.496332
vs      0.970035
am     -0.901186
gear   -0.201285
carb   -0.273598
dtype: float64

El modelo explica 87.5% de la variación en qsec. Un mayor peso y motores en línea (vs=1) aumentan el tiempo, mientras que más cilindros, potencia o marchas tienden a reducirlo.

## Train-test y Ridge para qsec

In [6]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, train_size=0.4, random_state=0)

lin2 = LinearRegression().fit(X2_train, y2_train)
print("R² train:", round(r2_score(y2_train, lin2.predict(X2_train)),4))
print("R² test :", round(r2_score(y2_test, lin2.predict(X2_test)),4))

for a in [0.01, 0.1, 1, 10, 100]:
    ridge2 = Ridge(alpha=a).fit(X2_train, y2_train)
    print(f"alpha={a:>5}: R² train={r2_score(y2_train, ridge2.predict(X2_train)):.3f},",
          f"R² test={r2_score(y2_test, ridge2.predict(X2_test)):.3f}")


R² train: 0.9975
R² test : 0.3432
alpha= 0.01: R² train=0.997, R² test=0.409
alpha=  0.1: R² train=0.993, R² test=0.611
alpha=    1: R² train=0.941, R² test=0.690
alpha=   10: R² train=0.741, R² test=0.493
alpha=  100: R² train=0.640, R² test=0.392


El modelo inicial sobreajusta; con α ≈ 1 se mejora la generalización, aumentando R² en test sin perder mucho en train.

## Ejercicio 2.1 (mpg con dummies en cyl, gear y carb)

In [7]:
data3 = data.copy()
X3 = pd.get_dummies(data3.drop(columns=['mpg']), columns=['cyl','gear','carb'], drop_first=True)
y3 = data3['mpg']

modelo3 = LinearRegression().fit(X3, y3)
print("R² completo:", round(r2_score(y3, modelo3.predict(X3)), 4))

pd.Series(modelo3.coef_, index=X3.columns).sort_values(ascending=False)


R² completo: 0.8931


carb_8    7.250411
carb_6    4.477569
carb_3    2.999639
gear_5    2.528396
vs        1.930851
am        1.212116
drat      1.182830
gear_4    1.114355
carb_4    1.091423
qsec      0.367845
disp      0.035546
hp       -0.070507
cyl_8    -0.336163
carb_2   -0.979354
cyl_6    -2.648695
wt       -4.529776
dtype: float64

El R² completo de 0.8931 indica que el modelo explica bien la variabilidad del mpg. Las variables más influyentes positivamente son carb_8, carb_6 y carb_3, mientras que wt y cyl_6 tienen el mayor efecto negativo sobre mpg.

## Train-test para mpg con dummies

In [8]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, train_size=0.4, random_state=0)

lin3 = LinearRegression().fit(X3_train, y3_train)
print("R² train:", round(r2_score(y3_train, lin3.predict(X3_train)),4))
print("R² test :", round(r2_score(y3_test, lin3.predict(X3_test)),4))


R² train: 1.0
R² test : -0.1589


El modelo está completamente sobreajustado: predice perfectamente los datos de entrenamiento (R² train = 1.0), pero falla totalmente en el conjunto de prueba (R² test negativo), mostrando que no generaliza en absoluto.

## Ejercicio 2.2 (qsec con dummies) 

In [9]:
X4 = pd.get_dummies(data.drop(columns=['qsec']), columns=['cyl','gear','carb'], drop_first=True)
y4 = data['qsec']

modelo4 = LinearRegression().fit(X4, y4)
print("R² completo:", round(r2_score(y4, modelo4.predict(X4)), 4))

pd.Series(modelo4.coef_, index=X4.columns).sort_values(ascending=False)


R² completo: 0.9083


gear_4    1.332282
wt        0.810472
vs        0.265732
gear_5    0.182317
drat      0.107866
mpg       0.027741
disp      0.003531
hp       -0.002066
carb_3   -0.229304
carb_2   -0.834413
cyl_6    -1.104343
carb_8   -1.332317
carb_6   -1.565241
am       -1.694294
carb_4   -1.947034
cyl_8    -2.966901
dtype: float64

El modelo tiene un R² alto (0.9083), mostrando buen ajuste general. Variables como gear_4, wt y vs aumentan el mpg, mientras que cyl_8, carb_4 y am tienen el mayor efecto negativo.

## Train-test para qsec con dummies

In [10]:
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, train_size=0.4, random_state=0)

lin4 = LinearRegression().fit(X4_train, y4_train)
print("R² train:", round(r2_score(y4_train, lin4.predict(X4_train)),4))
print("R² test :", round(r2_score(y4_test, lin4.predict(X4_test)),4))


R² train: 1.0
R² test : 0.6419


El modelo ajusta perfectamente los datos de entrenamiento (R² train = 1.0) y muestra una generalización moderada en prueba (R² test = 0.6419), indicando que aunque captura bien los patrones, aún podría sobreajustarse ligeramente.

## Comparaciones finales (3.1 y 3.2)

In [11]:
print("Comparación mpg: 1.1 vs 2.1")
print("R² sin dummies:", round(r2_score(y, modelo.predict(X)),4))
print("R² con dummies:", round(r2_score(y3, modelo3.predict(X3)),4))

print("\nComparación qsec: 1.2 vs 2.2")
print("R² sin dummies:", round(r2_score(y2, modelo2.predict(X2)),4))
print("R² con dummies:", round(r2_score(y4, modelo4.predict(X4)),4))


Comparación mpg: 1.1 vs 2.1
R² sin dummies: 0.869
R² con dummies: 0.8931

Comparación qsec: 1.2 vs 2.2
R² sin dummies: 0.8747
R² con dummies: 0.9083


Agregar variables dummies mejora ligeramente el modelo en ambos casos: para mpg el R² sube de 0.869 a 0.8931, y para qsec de 0.8747 a 0.9083, indicando que las categorías capturan información adicional que ayuda a explicar la variabilidad de los datos.